**PAQUETERIAS NECESARIAS**

In [1]:
## INCLUYE TODOS LOS PAQUETES A USAR
import tensorflow as tf
import random
from tensorflow import keras
 # Bibliotecas de ayuda
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import gaussian
 # Biblioteca para datos de entrenamiento
from sklearn.model_selection import train_test_split
import h5py as h # manipular extensiones
# FUNCIONES DEFINIDAS PARA USO NECESARIO
#import PAQ

**ENTRADA DE LA INFORMACION, ORDENAMIENTO DE LA INFORMACION A USAR**

In [2]:
 # ENTRAR LA INFORMACION DESDE UN ARCHIVO EXTERNO
FILE  = h.File("DATOS/TODOS.h5", 'r')
INPUT = np.vstack([FILE["qcd/images"][:], FILE["wprime/images"][:]]); INPUT=INPUT/np.max(INPUT) # SE NORMALIZA
LABEL = np.hstack([FILE["qcd/labels"][:], FILE["wprime/labels"][:]])
OUTPUT_LINEAL = np.ones((len(LABEL), 2));
OUTPUT_LINEAL[:,0] = LABEL < 1    ;     OUTPUT_LINEAL[:,1] = LABEL > 0
 # REACONDICIONAR LA INFORMACION INPUT EN UNA MATRIX PARA SU RAPIDO ACCESO
INPUT_LINEAL = np.reshape(INPUT, [len(INPUT),-1]) # LA INFORMACION SE LINEALIZA Y SE LOCALIZA EN LAS FILAS
 # SEPARAR LA INFO PARA ENTRENAR DE LA QUE SERA USADA PARA EL TEST DE CORRESPONDENCIA
INPUT_LINEAL, INPUT_LINEAL_TEST, OUTPUT_LINEAL, OUTPUT_LINEAL_TEST = train_test_split( 
    INPUT_LINEAL, OUTPUT_LINEAL, test_size=0.2, shuffle = True);

**PARAMETROS INICIALES DE ENTRENAMIENTO**

In [3]:

# def neuronas(v_input, v_output, n_capas , exp ):
def neuronas(v_input, v_output, exp = 1, n_capas = 2, modo = 'poly' ):
    x = np.ones(n_capas + 2)
    if  exp >= 1: #modo == 'poly':
        for i in range(n_capas + 2):
            x[len(x)-i-1] = round(( v_input - v_output )*pow(i/(n_capas + 1 ), exp) + v_output)
            #print(x[i])
            #print(i)
    elif exp < 1:
        for i in range(n_capas + 2):
            x[len(x)-i-1] = round(( v_input - v_output )*pow(i/(n_capas + 1 ), 1/exp) + v_output)
            #print(x[i])
            #print(i)
    return x

 # print(neuronas(100,2,3))
n=np.int64(neuronas(len(INPUT_LINEAL[0,:]),2,1))
#print(n)
learning_rate = 1e-4;   n_iterations = 3000   ;   dropout = 0.5
 # print(range(len(n)-1))
for i in range(len(n)-1):
    #print(n[i])
    #print(i)
    weights = tf.Variable(tf.truncated_normal([n[i], n[i+1]], stddev=0.1), name=f"w{i}")
    biases = tf.Variable(tf.constant(0.1, shape=[n[i+1]]), name=f"b{i}")
    if  i==0: # CONDICION INICIAL
        X = tf.compat.v1.placeholder("float", [None, n[0]]); # Estructura de la entrada
        layer = tf.add(tf.matmul(X, weights), biases, name=f"layer{i}")
    #elif i==len(n)-2: # CONDICION FINAL
    #    print(tf.add(tf.matmul(layer, weights),biases))
    #    layer = tf.matmul(layer, weights) + biases  
    else: # VALORES INTERNMEDIOS
        layer = tf.add(tf.matmul(layer, weights), biases, name=f"layer{i}")
    
Y = tf.compat.v1.placeholder("float", [None, n[len(n)-1]]) #Estructura de la salida
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=layer))
train_step = tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

correct_pred = tf.equal(tf.argmax(layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

**INICIALIZAR EL ENTRENAMIENTO**

In [4]:
init = tf.compat.v1.global_variables_initializer()

cost_summary = tf.compat.v1.summary.scalar("Cost", cross_entropy)
acc_summary = tf.compat.v1.summary.scalar("Accuracy", accuracy)
all_summary = tf.compat.v1.summary.merge_all()#'''

**ENTRENAMIENTO**

In [7]:
#saver=tf.train.Saver() #SALVAR LA SECCION PARA PODERLA UTILIZAR LUEGO
with tf.Session() as sess:
    writer = tf.summary.FileWriter("Tensorboard", sess.graph)
    #print(sess)
    sess.run(init)
    #saver.save(sess, 'modelo')
    for i in range(n_iterations):
        sess.run(train_step, feed_dict={X: INPUT_LINEAL, Y:OUTPUT_LINEAL}) # CORRIDA
        
        summary_results, loss, acc = sess.run([all_summary, cross_entropy, accuracy], feed_dict={X: INPUT_LINEAL, Y: OUTPUT_LINEAL})
        writer.add_summary(summary_results, i)
        if (i)%10 == 0:
            print("Iteration", str(i), "\t| Loss =", str(loss), "\t| Accuracy =", str(acc))
    
        if acc>.70:
            print("Iteration", str(i), "\t| Loss =", str(loss), "\t| Accuracy =", str(acc))
            break
    test_accuracy = sess.run(accuracy, feed_dict={X: INPUT_LINEAL_TEST, Y: OUTPUT_LINEAL_TEST})
    print("\nAccuracy on test set:", test_accuracy)
    
    a, b = sess.run([tf.argmax(layer, 1),tf.argmax(Y,1)], feed_dict={X: INPUT_LINEAL, Y: OUTPUT_LINEAL}) 
    
    for ea, eb in zip(a, b):
        print(ea,eb)
    #saver.save(sess, 'modelo') #SALVAR EL MODELO EXTERNAMENTE'''

Iteration 0 	| Loss = 0.69107914 	| Accuracy = 0.60026675
Iteration 10 	| Loss = 0.6641551 	| Accuracy = 0.631609
Iteration 20 	| Loss = 0.6422833 	| Accuracy = 0.60826904
Iteration 30 	| Loss = 0.6211474 	| Accuracy = 0.66761935
Iteration 40 	| Loss = 0.6006909 	| Accuracy = 0.6916262
Iteration 43 	| Loss = 0.5946828 	| Accuracy = 0.70239115

Accuracy on test set: 0.6982857
1 1
0 0
1 1
1 1
0 0
1 1
1 1
0 0
1 1
1 1
0 0
1 1
1 1
1 0
0 0
1 1
1 1
1 1
1 1
1 0
1 1
1 0
1 1
0 0
0 0
1 1
1 0
1 1
1 1
1 1
1 1
1 1
1 1
1 0
1 0
1 1
1 1
1 1
0 0
0 0
1 1
1 1
1 1
1 0
1 1
1 1
1 1
0 0
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 1
1 0
1 1
1 1
1 0
1 0
1 1
1 1
0 0
1 0
1 1
1 1
0 0
1 1
1 0
1 1
1 0
1 1
1 1
1 1
0 0
1 1
0 0
1 0
1 0
1 0
1 1
1 1
1 1
1 1
1 1
1 1
1 0
1 0
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 1
0 0
1 1
1 1
0 0
1 0
1 1
1 0
1 0
1 1
0 0
1 0
1 0
1 0
1 1
1 1
1 1
1 1
1 0
1 0
1 1
1 1
1 1
1 0
1 0
1 1
1 1
1 1
1 0
1 0
1 0
1 0
1 1
0 0
1 1
0 0
1 0
0 0
1 1
1 0
1 0
0 0
1 1
1 1
1 0
0 0
1 1
1 0
1 0
1 0
0 0
1 1
1 1
1 1
1 1
1 0
1 1
0 0
1 1
1 

1 1
1 1
1 1
1 0
1 0
1 0
1 1
1 1
1 0
1 0
1 1
1 1
1 1
1 0
1 1
1 0
1 1
1 0
1 1
1 1
1 1
1 1
1 1
1 1
1 1
0 0
1 1
1 0
1 0
1 1
1 1
1 0
1 1
1 1
1 0
1 0
1 1
1 1
1 1
1 1
1 1
1 1
1 0
1 0
1 1
1 1
1 1
1 0
1 1
1 1
0 0
1 0
1 1
1 1
0 0
1 1
0 0
1 1
1 0
1 1
1 1
1 0
1 0
1 0
1 0
1 1
1 1
1 0
1 1
0 0
1 0
1 1
1 0
1 0
1 0
1 0
1 1
0 0
1 1
1 1
1 1
0 0
1 0
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 0
1 0
1 0
1 1
0 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 1
1 1
1 0
1 1
1 0
1 0
0 1
0 0
1 1
1 1
1 1
0 0
1 1
0 0
1 1
1 1
1 0
1 0
0 0
1 0
1 0
1 1
1 1
0 0
1 0
1 1
1 0
1 1
1 1
1 0
1 1
1 1
1 0
1 1
1 0
1 1
1 0
0 0
0 0
1 1
1 1
1 0
1 0
1 1
1 1
1 1
1 0
1 1
1 0
1 0
1 1
0 0
1 1
1 1
1 1
0 0
1 0
1 0
0 0
1 0
1 1
1 1
0 0
1 1
1 1
1 0
1 1
1 0
0 1
1 1
1 1
1 1
1 0
1 1
1 0
1 1
1 0
1 0
1 1
1 1
1 1
1 0
1 0
1 1
1 1
1 0
1 0
0 0
1 0
1 1
0 0
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 1
1 1
1 1
0 0
0 0
1 1
1 0
1 0
1 1
1 1
1 1
1 1
0 0
1 0
1 1
1 0
1 1
1 1
1 1
0 0
1 0
1 0
1 1
1 1
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 0
1 1
1 1
0 0
1 1
1 1
0 0
1 1
1 1
1 0
1 1
1 1
1 0
0 0
1 1
1 1
1 1
1 1


1 0
0 0
1 1
1 1
0 0
1 1
1 1
1 1
1 1
1 0
0 0
1 0
1 1
1 1
1 1
1 1
1 0
0 0
0 0
0 0
1 0
1 1
1 1
1 1
0 0
1 1
1 1
1 0
1 1
1 1
1 1
0 0
1 1
0 0
1 0
1 1
1 1
1 1
1 1
1 0
1 0
1 1
1 1
1 0
1 0
0 0
1 1
0 0
1 1
0 0
1 0
1 0
1 1
1 1
1 1
1 1
1 1
1 1
0 0
1 1
1 1
1 1
1 0
1 1
1 0
0 0
1 0
1 1
1 1
1 1
1 1
1 0
1 1
1 0
0 0
1 1
1 1
1 0
1 0
0 0
1 0
1 1
0 0
1 1
1 1
1 0
0 0
1 1
1 1
1 1
1 1
1 1
1 0
1 0
1 0
1 0
1 1
1 1
1 1
1 0
1 1
1 0
1 0
1 1
1 1
1 0
0 0
1 0
1 0
1 1
0 0
1 0
1 1
1 1
1 1
0 0
1 1
1 0
1 1
0 0
1 1
0 0
1 1
1 0
0 0
1 0
1 0
1 1
1 0
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
0 0
1 0
1 0
1 0
1 1
1 0
1 0
0 0
1 0
1 0
1 0
1 1
1 1
1 1
1 0
1 1
0 0
1 1
1 1
1 0
1 1
1 0
0 0
1 0
1 1
1 1
0 0
1 0
1 1
1 1
1 0
1 1
1 1
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 0
1 1
1 0
1 0
1 0
1 1
1 0
1 1
1 1
1 1
1 1
1 0
1 1
1 0
1 1
1 1
1 1
0 0
0 0
1 0
1 1
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 1
1 1
1 1
1 0
1 1
1 0
1 1
1 0
1 1
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 0
1 1
1 1
1 0
1 1
1 1
1 1
1 1
1 1
0 0
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
0 0
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
0 0


1 1
0 0
1 0
1 1
1 0
1 1
1 1
1 1
1 0
1 0
1 1
1 1
1 1
1 0
0 0
1 0
1 1
1 1
1 1
1 1
1 1
1 1
1 0
1 0
1 0
1 1
1 1
1 1
1 1
0 0
1 0
1 0
1 1
1 1
1 0
1 0
1 1
0 0
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 0
1 1
1 1
1 0
1 0
1 0
1 0
1 1
1 1
1 0
1 0
1 0
1 0
1 1
1 1
1 1
1 1
0 0
1 0
1 0
1 0
1 0
1 1
1 0
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 0
1 1
0 0
1 0
1 1
1 1
1 1
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 1
1 0
1 1
1 0
1 1
1 1
0 0
1 1
1 1
1 1
1 0
1 1
1 1
1 0
1 1
1 0
1 1
1 1
1 0
1 1
1 1
1 1
1 0
0 0
1 1
1 0
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 1
1 0
1 0
1 1
1 0
1 1
1 1
1 1
1 1
1 1
0 0
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 1
1 0
1 1
1 1
0 0
1 1
1 1
1 1
0 0
1 0
1 1
1 0
1 1
1 1
1 0
1 1
1 1
1 1
0 0
1 1
0 0
1 1
0 0
1 1
1 1
1 0
1 1
1 1
1 1
1 0
1 0
1 1
1 1
1 1
1 0
1 1
1 1
1 0
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 0
1 1
1 1
1 0
1 1
1 1
1 1
0 0
1 1
1 1
1 0
1 1
1 1
1 0
1 1
0 0
1 0
0 0
1 1
1 1
1 1
1 1
0 0
1 1
0 0
1 1
1 1
1 1
1 0
1 0
1 1
1 0
1 1
1 1
1 1
0 0
1 1
1 1
1 0
0 0
1 0
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 0


0 0
0 0
1 0
1 1
1 1
1 0
1 1
1 1
0 0
0 0
1 0
0 0
1 0
1 1
1 1
1 1
1 1
1 1
1 0
1 0
1 1
1 1
1 0
1 0
1 1
1 0
1 1
1 0
1 0
1 0
1 0
1 1
1 1
1 1
1 1
1 1
0 0
1 1
1 1
1 0
1 0
1 0
0 0
1 0
1 0
0 0
1 1
1 0
1 1
1 1
1 1
1 1
0 0
1 1
1 0
1 0
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 0
1 1
1 0
1 1
1 1
0 0
1 1
1 1
1 0
1 1
1 0
1 1
1 0
1 0
1 1
1 1
1 0
1 0
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 0
1 1
1 0
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 1
0 0
1 1
1 0
1 0
1 1
1 1
1 0
1 1
1 1
1 1
0 0
1 0
1 1
1 0
1 1
1 1
1 1
1 1
1 1
1 0
0 0
1 0
1 1
0 0
1 1
0 0
1 1
1 1
0 0
1 1
1 0
1 1
0 0
1 0
1 1
1 1
1 1
1 0
1 1
1 0
1 1
1 0
1 1
1 1
1 1
0 0
1 1
1 1
1 0
1 0
1 1
1 0
1 0
1 1
1 0
0 0
1 1
1 0
1 1
0 0
1 0
1 1
1 1
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 1
1 1
0 0
1 1
1 0
1 0
1 1
1 0
1 0
1 0
1 1
1 1
1 1
1 0
1 0
1 1
1 1
1 0
1 1
1 1
1 1
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 1
1 1
1 1
1 0
0 0
1 0
1 0
1 1
1 1
1 0
1 1
1 1
1 0
1 1
1 0
1 1
1 0
1 1
1 0
1 0
1 1
1 0
1 1
1 0
1 1
1 0
1 1
1 0
1 0
1 0
1 1
1 1
1 0
1 0
1 1
1 1
1 1
1 1
1 0
1 0
1 1
1 0
1 1
1 0
1 0
1 1
1 0
0 0
1 0
0 0
1 1
1 1
1 0


**PAQUETERIAS NECESARIAS**